Check we have connection correct. To test this we call get a FHIR Conformance statement from the server

In [1]:
import fhirclient
import requests
from fhirclient.models.fhirinstant import FHIRInstant
from requests.auth import HTTPBasicAuth
from dotenv import load_dotenv
load_dotenv()
import os

fhir_password = os.getenv("FHIR_PASSWORD")
fhir_username = os.getenv("FHIR_USERNAME")
#server = "https://gen-tie-test.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/"
server = os.getenv("FHIR_SERVER")

api_url = server + "metadata"
print(api_url)
response = requests.get(api_url)
#response.json()

https://gen-tie-test.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/metadata


Find a patient. Simple search for a patient named wrexham.

The data used here is a combination of patient demographics from EHR systems (supplied with the orders) and [NHS England Personal Demographics Service - FHIR API](https://digital.nhs.uk/developer/api-catalogue/personal-demographics-service-fhir)

In [2]:


api_url = server + "Patient?name=wrexham"
print(api_url)
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
patientJSON = response.json()
print(patientJSON)

patientId = None


if ((patientJSON['total']> 0) and (len(patientJSON['entry'])>0)):
    patientId = patientJSON['entry'][0]['resource']['id']
    print()
    print("Patient = " + patientId)

https://gen-tie-test.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/Patient?name=wrexham
{'resourceType': 'Bundle', 'id': '23cfb4b1-6878-453a-b0d6-3a254d3c175d', 'type': 'searchset', 'timestamp': '2025-12-26T06:59:29Z', 'total': 0, 'link': [{'relation': 'self', 'url': 'https://gen-tie-test.nwgenomics.nhs.uk/dataplatform/cdr/fhir/r4/Patient?name=wrexham'}]}


Now lets find diagnostic reports for this patient.

In [3]:
organisationId = None

if patientId != None:
    api_url = server + "DiagnosticReport?patient="+"Patient/"+patientId
    response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
    diagnosticReportsJSON = response.json()
    print(diagnosticReportsJSON)
    if (len(diagnosticReportsJSON['entry'])>0):
        organisationId = diagnosticReportsJSON['entry'][0]['resource']['performer'][0]['reference'].replace('Organization/', '')
        print()
        print("Organization = " + organisationId)


Can also return details about the organisation that created this.

This data is sourced from [NHS England Organisation Data Terminology - FHIR API](https://digital.nhs.uk/developer/api-catalogue/organisation-data-terminology). Note: the model used here is simplified.

In [4]:
if organisationId != None:
    api_url = server + "Organization/"+organisationId
    print(api_url)
    response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
    organisationJSON = response.json()
    print(organisationJSON)

Example using python to interpret the json and also using [fhirclient](https://github.com/smart-on-fhir/client-py)

In [5]:
import fhirclient.models.diagnosticreport as dr
import pandas as pd

api_url = server + "DiagnosticReport?_count=50"
response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
response1JSON = response.json()

print(response1JSON['total'])

reports = []
for entry in response1JSON['entry']:
    #print(entry['resource'])
    print(entry['resource']['resourceType'], entry['resource']['issued'] )
    report = dr.DiagnosticReport(entry['resource'])
    reports.append(report)
    for coding in report.code.coding:
        print(coding.code)

10
DiagnosticReport 2025-12-18T10:13:19+00:00
CCP19v1_R112
R112.1
DiagnosticReport 2025-11-21T12:05:26+00:00
CCP19v1_R112
R112.1
DiagnosticReport 2025-11-21T12:22:53+00:00
CCP19v1_R112
R112.1
R115.1
DiagnosticReport 2025-11-21T12:42:05+00:00
CCP19v1_R112
R112.1
R115.1
DiagnosticReport 2025-11-21T12:53:09+00:00
CCP19v1_R112
R112.1
DiagnosticReport 2025-11-21T13:47:18+00:00
CCP19v1_R112
R112.1
R115.1
DiagnosticReport 2025-11-21T14:01:59+00:00
CCP19v1_R112
R112.1
DiagnosticReport 2025-11-21T15:34:09+00:00
CCP19v1_R112
R112.1
R115.1
DiagnosticReport 2025-11-25T14:15:15+00:00
CCP19v1_R112
R112.1
DiagnosticReport 2025-11-27T10:29:01+00:00
ctDNA_M4
M4.14


We are aiming at display graphs based on the order and the report.

We can include the order which in FHIR is held in ServiceRequest by including this in the query.

In [6]:
import fhirclient.models.servicerequest as sr

serviceRequests = []
diagnosticReports = []

api_url = server + "DiagnosticReport?_include=DiagnosticReport:based-on"


while True:
    response = requests.get(api_url, auth=HTTPBasicAuth(fhir_username, fhir_password))
    responseInclude = response.json()

    #print(responseInclude)
    print(responseInclude['total'])
    entry = responseInclude['entry']
    print(len(entry))
    if len(entry) == 0:
        break
    for entry in responseInclude['entry']:

        if entry['resource']['resourceType'] == 'DiagnosticReport':
            report = dr.DiagnosticReport(entry['resource'])
            diagnosticReports.append(report)
        if entry['resource']['resourceType'] == 'ServiceRequest':
            request = sr.ServiceRequest(entry['resource'])
            serviceRequests.append(request)

    print("ServiceRequest = " + str(len(serviceRequests)))
    print("DiagnosticReport = " + str(len(diagnosticReports)))


    found = False
    for link in responseInclude['link']:
        if link['relation'] == 'next':
            api_url = link['url']
            found = True
            print(api_url)
    if found == False:
        break

10
19
ServiceRequest = 9
DiagnosticReport = 10


Process the diaganostic reports

Process the reports, convert some of the objects into other datatypes.

In [7]:
import fhirclient.models.meta as meta
from dateutil import parser

def performer(my_list):
    performr = ""
    if my_list != None:
        for item in my_list:
            performr = item.display
    return performr

def performerCode(my_list):
    performr = None
    if my_list != None:
        for item in my_list:
            performr = item.identifier.value
    return performr

def codeCode(concept):
    code = ""
    for coding in concept.coding:
        code = coding.code

    return code
def codeDisplay(concept):
    code = ""
    for coding in concept.coding:
        code = coding.display

    return code

def issued(issued):
    if issued == None:
        return None
    return parser.parse(issued.isostring)

def serviceRequest(my_list):
    sr = None
    if my_list != None:
        for item in my_list:
            if item.reference != None:
                sr = item.reference.replace('ServiceRequest/', '')
    return sr
def lastUpdated(meta : meta.Meta):
    if meta == None:
        return None
    return parser.parse(meta.lastUpdated.isostring)

print(len(diagnosticReports))
dfDR = pd.DataFrame([vars(s) for s in diagnosticReports])

dfDR['performerDisplay'] = dfDR['performer'].apply(performer)
dfDR['performerCode'] = dfDR['performer'].apply(performerCode)
dfDR['codingCode'] = dfDR['code'].apply(codeCode)
dfDR['codingDisplay'] = dfDR['code'].apply(codeDisplay)
dfDR['lastUdatedDate'] = dfDR['meta'].apply(lastUpdated)
dfDR['issuedDate'] = dfDR['issued'].apply(issued)
dfDR['effectiveDate'] = dfDR['effectiveDateTime'].apply(issued)
dfDR['serviceRequestId'] = dfDR['basedOn'].apply(serviceRequest)

dfDiagnosticReport = dfDR[['id','performerDisplay','performerCode','codingCode', 'codingDisplay', 'lastUdatedDate','issuedDate', 'effectiveDate', 'serviceRequestId']]
dfDiagnosticReport

10


,id,performerDisplay,performerCode,codingCode,codingDisplay,lastUdatedDate,issuedDate,effectiveDate,serviceRequestId
0,6,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-12-25 08:08:56+00:00,2025-12-18 10:13:19+00:00,2025-12-18 10:13:19+00:00,81
1,16,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-12-25 07:41:29+00:00,2025-11-21 12:05:26+00:00,2025-11-21 12:05:26+00:00,None
2,21,NHS North West Genomics,699X0,R115.1,Factor V deficiency (Single gene sequencing >=...,2025-12-25 08:08:28+00:00,2025-11-21 12:22:53+00:00,2025-11-21 12:22:53+00:00,74
3,27,NHS North West Genomics,699X0,R115.1,Factor V deficiency (Single gene sequencing >=...,2025-12-25 08:08:30+00:00,2025-11-21 12:42:05+00:00,2025-11-21 12:42:05+00:00,76
4,31,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-12-25 08:08:35+00:00,2025-11-21 12:53:09+00:00,2025-11-21 12:53:09+00:00,30
5,39,NHS North West Genomics,699X0,R115.1,Factor V deficiency (Single gene sequencing >=...,2025-12-25 08:08:37+00:00,2025-11-21 13:47:18+00:00,2025-11-21 13:47:18+00:00,36
6,43,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-12-25 08:08:40+00:00,2025-11-21 14:01:59+00:00,2025-11-21 14:01:59+00:00,42
7,49,NHS North West Genomics,699X0,R115.1,Factor V deficiency (Single gene sequencing >=...,2025-12-25 08:08:44+00:00,2025-11-21 15:34:09+00:00,2025-11-21 15:34:09+00:00,83
8,53,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-12-25 08:08:47+00:00,2025-11-25 14:15:15+00:00,2025-11-25 14:15:15+00:00,52
9,61,NHS North West Genomics,699X0,M4.14,"Non-Small Cell Lung Cancer, Multi-target ctDNA...",2025-12-25 08:08:48+00:00,2025-11-27 10:29:01+00:00,2025-11-27 10:29:01+00:00,87


Clean up requests

In [8]:
print(len(serviceRequests))

def requester(item):
    performr = None
    if item != None:
        performr = item.display
    return performr

def requesterCode(item):
    performr = None
    if item != None:
        performr = item.identifier.value
    return performr

dfSR = pd.DataFrame([vars(s) for s in serviceRequests])
dfSR['requesterDisplay'] = dfSR['requester'].apply(requester)
dfSR['requesterCode'] = dfSR['requester'].apply(requesterCode)
dfSR['authoredOnDate'] = dfSR['authoredOn'].apply(issued)

dfServiceRequest = dfSR[['id','requesterDisplay','requesterCode','authoredOnDate']]
dfServiceRequest

9


,id,requesterDisplay,requesterCode,authoredOnDate
0,74,Bolton NHS Foundation Trust,RMC,2025-11-06
1,76,Stockport NHS Foundation Trust,RWJ,2025-11-06
2,30,Manchester University NHS Foundation Trust,R0A,2025-11-06
3,36,Manchester University NHS Foundation Trust,R0A,2025-11-06
4,42,Manchester University NHS Foundation Trust,R0A,2025-11-06
5,83,Bolton NHS Foundation Trust,RMC,2025-11-06
6,52,Manchester University NHS Foundation Trust,R0A,2025-11-03
7,81,Liverpool Women's NHS Foundation Trust,REP,2025-11-06
8,87,Unknown,UNK,2025-11-05


Join both dataframes into a single result.

In [9]:
# ... existing code ...
df = pd.merge(
    dfDiagnosticReport,
    dfServiceRequest,
    left_on='serviceRequestId',
    right_on='id',
    how="left",
    indicator=True,
    suffixes=('_dr', '_sr')
)

df

,id_dr,performerDisplay,performerCode,codingCode,codingDisplay,lastUdatedDate,issuedDate,effectiveDate,serviceRequestId,id_sr,requesterDisplay,requesterCode,authoredOnDate,_merge
0,6,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-12-25 08:08:56+00:00,2025-12-18 10:13:19+00:00,2025-12-18 10:13:19+00:00,81,81,Liverpool Women's NHS Foundation Trust,REP,2025-11-06,both
1,16,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-12-25 07:41:29+00:00,2025-11-21 12:05:26+00:00,2025-11-21 12:05:26+00:00,None,NaN,NaN,NaN,NaT,left_only
2,21,NHS North West Genomics,699X0,R115.1,Factor V deficiency (Single gene sequencing >=...,2025-12-25 08:08:28+00:00,2025-11-21 12:22:53+00:00,2025-11-21 12:22:53+00:00,74,74,Bolton NHS Foundation Trust,RMC,2025-11-06,both
3,27,NHS North West Genomics,699X0,R115.1,Factor V deficiency (Single gene sequencing >=...,2025-12-25 08:08:30+00:00,2025-11-21 12:42:05+00:00,2025-11-21 12:42:05+00:00,76,76,Stockport NHS Foundation Trust,RWJ,2025-11-06,both
4,31,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-12-25 08:08:35+00:00,2025-11-21 12:53:09+00:00,2025-11-21 12:53:09+00:00,30,30,Manchester University NHS Foundation Trust,R0A,2025-11-06,both
5,39,NHS North West Genomics,699X0,R115.1,Factor V deficiency (Single gene sequencing >=...,2025-12-25 08:08:37+00:00,2025-11-21 13:47:18+00:00,2025-11-21 13:47:18+00:00,36,36,Manchester University NHS Foundation Trust,R0A,2025-11-06,both
6,43,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-12-25 08:08:40+00:00,2025-11-21 14:01:59+00:00,2025-11-21 14:01:59+00:00,42,42,Manchester University NHS Foundation Trust,R0A,2025-11-06,both
7,49,NHS North West Genomics,699X0,R115.1,Factor V deficiency (Single gene sequencing >=...,2025-12-25 08:08:44+00:00,2025-11-21 15:34:09+00:00,2025-11-21 15:34:09+00:00,83,83,Bolton NHS Foundation Trust,RMC,2025-11-06,both
8,53,NHS North West Genomics,699X0,R112.1,Factor II deficiency (Single gene sequencing >...,2025-12-25 08:08:47+00:00,2025-11-25 14:15:15+00:00,2025-11-25 14:15:15+00:00,52,52,Manchester University NHS Foundation Trust,R0A,2025-11-03,both
9,61,NHS North West Genomics,699X0,M4.14,"Non-Small Cell Lung Cancer, Multi-target ctDNA...",2025-12-25 08:08:48+00:00,2025-11-27 10:29:01+00:00,2025-11-27 10:29:01+00:00,87,87,Unknown,UNK,2025-11-05,both


In [10]:
import plotly.graph_objects as go
import ipykernel

fig = go.Figure()
dfTotals = df.requesterCode.value_counts().reset_index(name='counts')
fig.add_trace(go.Bar(x=dfTotals['requesterCode'], y=dfTotals['counts']))
fig.show()

fig2 = go.Figure()
dfTotals2 = df.codingCode.value_counts().reset_index(name='counts')
fig2.add_trace(go.Bar(x=dfTotals2['codingCode'], y=dfTotals['counts']))
fig2.show()

